In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [2]:
movies = pd.read_csv(
    "./Datasets/tmdb movie database/tmdb_5000_movies.csv",
    index_col="id"
)

In [3]:
credits = pd.read_csv(
    "./Datasets/tmdb movie database/tmdb_5000_credits.csv",
    index_col="movie_id"
)

## Genres

In [4]:
genres: dict[int, str] = dict()
movie_genre: list[tuple] = list()
for movie_id, row in movies.loc[:, ["genres", "title"]].iterrows():
    genre_list: list[dict] = literal_eval(row["genres"])
    for genre in genre_list:
        # if genre not in dictionary, add it with the name
        genres.setdefault(genre["id"], genre["name"])

        # associate movie and genre in the movie_genre list
        movie_genre.append(
            (movie_id, row["title"], genre["id"], genre["name"])
        )

In [5]:
# remove genres from movies
movies.drop("genres", axis = "columns", inplace=True, errors="ignore")

# create genres table
genres_df = pd.DataFrame(data = genres.items(), columns=["id", "name"])

# create movie_genre table
movie_genre_df = pd.DataFrame(
    data = movie_genre,
    columns = ["movie_id", "movie_title", "genre_id", "genre_name"]
)

## Production Companies

In [6]:
production_companies: dict[int, str] = dict()
movie_company: list[tuple] = list()
columns = ["production_companies", "title"]
for movie_id, row in movies.loc[:, columns].iterrows():
    companies_list: list[dict] = literal_eval(row["production_companies"])
    for company in companies_list:
        # if company not in dictionary, add it with the name
        production_companies.setdefault(company["id"], company["name"])

        # associate movie and company in the movie_company list
        movie_company.append(
            (movie_id, row["title"], company["id"], company["name"])
        )

In [7]:
# remove production_companies from movies
movies.drop(
    "production_companies",
    axis = "columns",
    inplace=True,
    errors="ignore"
)

# create production_companies table
production_companies_df = pd.DataFrame(
    data = production_companies.items(),
    columns=["id", "name"]
)

# create movie_company table
movie_company_df = pd.DataFrame(
    data = movie_company,
    columns = [
        "movie_id",
        "movie_title",
        "production_company_id",
        "production_company_name"
    ]
)

## Production Countries

In [8]:
country_to_continent = pd.read_csv(
    "./Datasets/continents2.csv",
    index_col = "alpha-2"
)
country_to_continent.loc["CS", "region"] = "Europe"

In [9]:
production_countries: dict[int, str] = dict()
movie_country: list[tuple] = list()
columns = ["production_countries", "title"]
for movie_id, row in movies.loc[:, columns].iterrows():
    countries_list: list[dict] = literal_eval(row["production_countries"])
    for country in countries_list:
        # if country not in dictionary, add it with the name
        production_countries.setdefault(country["iso_3166_1"], country["name"])

        # associate movie and country in the movie_country list
        movie_country.append((
            movie_id,
            row["title"],
            country["iso_3166_1"],
            country["name"],
            country_to_continent.loc[country["iso_3166_1"], "region"]   
        ))

In [10]:
# remove production_countries from movies
movies.drop(
    "production_countries",
    axis = "columns",
    inplace=True,
    errors="ignore"
)

# create production_countries table
production_countries_df = pd.DataFrame(
    data = production_countries.items(),
    columns=["iso_3166_1", "name"]
)

# create movie_country table
movie_country_df = pd.DataFrame(
    data = movie_country,
    columns = [
        "movie_id",
        "movie_title",
        "production_country_iso_3166_1",
        "production_country_name",
        "continent"
    ]
)

## Remove Unnecessary Columns

In [11]:
movies.drop("homepage", axis = "columns", inplace=True, errors="ignore")
movies.drop("overview", axis = "columns", inplace=True, errors="ignore")
movies.drop("tagline", axis = "columns", inplace=True, errors="ignore")
movies.drop("keywords", axis = "columns", inplace=True, errors="ignore")
movies.drop(
    "spoken_languages",
    axis = "columns",
    inplace=True,
    errors="ignore"
)

## Cast and Crew

In [12]:
people: dict[int, tuple] = dict()
movie_cast: list[tuple] = list()
movie_crew: list[tuple] = list()
for movie_id, row in credits.loc[:, ["cast", "crew", "title"]].iterrows():
    cast_list: list[dict] = literal_eval(row["cast"])
    for cast in cast_list:
        # if person not in dictionary, add it with the name
        gender = (
            {0: "Not Specified", 1: "Female", 2: "Male"}[cast["gender"]]
            if cast["gender"] is not None else None
        )
        people.setdefault(cast["id"], (cast["id"], cast["name"], gender))

        # associate movie and cast in the movie_cast list
        movie_cast.append(
            (
                movie_id,
                row["title"],
                cast["character"],
                cast["id"],
                cast["name"],
                gender,
                cast["order"]
            )
        )

    crew_list: list[dict] = literal_eval(row["crew"])
    for crew in crew_list:
        # if person not in dictionary, add it with the name
        gender = (
            {0: "Not Specified", 1: "Female", 2: "Male"}[cast["gender"]]
            if crew["gender"] is not None else None
        )
        people.setdefault(crew["id"], (crew["id"], crew["name"], gender))

        # associate movie and crew in the movie_crew list
        movie_crew.append(
            (
                movie_id,
                row["title"],
                crew["department"],
                crew["job"],
                crew["id"],
                crew["name"],
                gender
            )
        )

In [13]:
# create people table
people_df = pd.DataFrame(
    data = people.values(),
    columns=["id", "name", "gender"]
)

# create movie_cast table
movie_cast_df = pd.DataFrame(
    data = movie_cast,
    columns = [
        "movie_id",
        "movie_title",
        "character",
        "person_id",
        "name",
        "gender",
        "order"
    ]
)

# create movie_crew table
movie_crew_df = pd.DataFrame(
    data = movie_crew,
    columns = [
        "movie_id",
        "movie_title",
        "department",
        "job",
        "person_id",
        "name",
        "gender"
    ]
)

## Add Director Experience Column

In [14]:
movies = movies.reset_index()

In [15]:
directors = movie_crew_df.loc[movie_crew_df["job"] == "Director", :]
movie_director = movies.loc[:, ["id"]].merge(
    right = directors.loc[:, ["movie_id", "person_id"]],
    how = "left",
    left_on = "id",
    right_on = "movie_id"
)
movie_director.drop(columns = "movie_id", inplace=True)
movie_count_by_director = movie_director.groupby(["person_id"])["id"].count()

def experience_level_by_movie_count(movie_count: int):
    if movie_count < 5:
        return "low"
    elif movie_count < 10:
        return "medium"
    else:
        return "high"

experience_level = list()
for movie_id in movies["id"]:
    director_ids = movie_director.loc[
        movie_director["id"] == movie_id,
        "person_id"
    ]
    movie_count = movie_count_by_director.loc[director_ids.dropna()].max()
    experience_level.append(experience_level_by_movie_count(movie_count))
    

movies["director_experience_level"] = experience_level
movies["director_experience_level"].value_counts()

director_experience_level
low       3168
medium    1131
high       504
Name: count, dtype: int64

## Add is_good Column

In [16]:
movies["is_good"] = movies["vote_average"] >= 7

## Save All Tables

In [19]:
movies.to_csv("./Datasets/tmdb movie database cleaned/movies.csv", index=False)
genres_df.to_csv(
    "./Datasets/tmdb movie database cleaned/genres.csv",
    index=False
)
movie_genre_df.to_csv(
    "./Datasets/tmdb movie database cleaned/movie_genre.csv",
    index=False
)
production_companies_df.to_csv(
    "./Datasets/tmdb movie database cleaned/production_companies.csv",
    index=False
)
movie_company_df.to_csv(
    "./Datasets/tmdb movie database cleaned/movie_company.csv",
    index=False
)
production_countries_df.to_csv(
    "./Datasets/tmdb movie database cleaned/production_countries.csv",
    index=False
)
movie_country_df.to_csv(
    "./Datasets/tmdb movie database cleaned/movie_country.csv",
    index=False
)
people_df.to_csv(
    "./Datasets/tmdb movie database cleaned/people.csv",
    index=False
)
movie_cast_df.to_csv(
    "./Datasets/tmdb movie database cleaned/movie_cast.csv",
    index=False
)
movie_crew_df.to_csv(
    "./Datasets/tmdb movie database cleaned/movie_crew.csv",
    index=False
)